In [1]:
import pandas
import sqlite3

In [3]:
connection = sqlite3.connect('nearby_stars.db')
c = connection.cursor()

## Add full MH08 age table to database

In [3]:
age_tab = pandas.read_csv('mh08_agetable.csv')
age_tab.to_sql('mh08_agetable', connection)

ValueError: Table 'mh08_agetable' already exists.

In [4]:
c.execute('PRAGMA table_info(mh08_agetable);')
c.fetchall()

[(0, u'index', u'INTEGER', 0, None, 0),
 (1, u'HD', u'INTEGER', 0, None, 0),
 (2, u'GJ', u'TEXT', 0, None, 0),
 (3, u'OName', u'TEXT', 0, None, 0),
 (4, u'l_tau1', u'TEXT', 0, None, 0),
 (5, u'tau1', u'REAL', 0, None, 0),
 (6, u'l_tau2', u'TEXT', 0, None, 0),
 (7, u'tau2', u'REAL', 0, None, 0)]

## Create ages table

Create table with primary key <tt>ageid</tt> (age measurement ID) and foreign key <tt>sid</tt> linking this table to the main <tt>stars_FGK_nobinaries</tt> table. Having a separation <tt>ageid</tt> allows this table to hold multiple age measurements per star if they exist.

In [5]:
c.execute('DROP TABLE ages')

In [6]:
create_table = '''
CREATE TABLE ages (
ageid INTEGER PRIMARY KEY AUTOINCREMENT,
age REAL,
age_lo REAL,
age_hi REAL,
bibcode TEXT,
sid INTEGER,
CONSTRAINT sid 
    FOREIGN KEY (sid)
    REFERENCES stars_FGK_nobinaries(sid)
)
'''

In [7]:
c.execute(create_table)

In [8]:
c.execute('PRAGMA table_info(ages);')
c.fetchall()

[(0, u'ageid', u'INTEGER', 0, None, 1),
 (1, u'age', u'REAL', 0, None, 0),
 (2, u'age_lo', u'REAL', 0, None, 0),
 (3, u'age_hi', u'REAL', 0, None, 0),
 (4, u'bibcode', u'TEXT', 0, None, 0),
 (5, u'sid', u'INTEGER', 0, None, 0)]

## Create subquery connecting star in MH08 table to star in main simbad_FGK_nobinaries table

In [34]:
cmd = '''
SELECT stars_FGK_nobinaries.sid, mh08_agetable.OName, stars_FGK_nobinaries.main_id, 
'HD ' || mh08_agetable.hd, stars_FGK_nobinaries.hd, mh08_agetable.gj, LTRIM(LTRIM(stars_FGK_nobinaries.GJ,'GJ'))
FROM mh08_agetable
    INNER JOIN stars_FGK_nobinaries 
    ON LTRIM(LTRIM(stars_FGK_nobinaries.hd,'HD')) LIKE mh08_agetable.hd
    OR LTRIM(LTRIM(stars_FGK_nobinaries.GJ,'GJ')) LIKE mh08_agetable.gj
'''
c.execute(cmd)
rows = c.fetchall()
print len(rows)
for row in rows:
    print row

39
(0, u'epsilon Eri', u'* eps Eri', u'HD 22049', u'HD  22049', u'144', u'144')
(1, u'tau Cet', u'* tau Cet', u'HD 10700', u'HD  10700', u'71', u'71')
(2, u'82 Eri', u'* e Eri', u'HD 20794', u'HD  20794', u'139', u'139')
(3, u'delta Pav', u'* del Pav', u'HD 190248', u'HD 190248', u'780', u'780')
(8, u'61 Vir', u'*  61 Vir', u'HD 115617', u'HD 115617', u'506', u'506')
(9, u'zeta Tuc', u'* zet Tuc', u'HD 1581', u'HD   1581', u'17', u'17')
(10, u'HR 7722', u'HD 192310', u'HD 192310', u'HD 192310', u'785', u'785')
(13, u'CF UMa', u'HD 103095', u'HD 103095', u'HD 103095', u'451A', u'451')
(18, u'alpha Men', u'* alf Men', u'HD 43834', u'HD  43834', u'231', u'231')
(20, u'HR 6806', u'HD 166620', u'HD 166620', u'HD 166620', u'706', u'706')
(21, u'54 Psc', u'*  54 Psc', u'HD 3651', u'HD   3651', u'27A', u'27')
(27, u'HR 3384', u'HD  72673', u'HD 72673', u'HD  72673', u'309', u'309')
(30, u'HR 3259', u'HD  69830', u'HD 69830', u'HD  69830', u'302', u'302')
(36, u'HR 683', u'HD  14412', u'HD 1441

39 is the same number of stars I found when querying Simbad with the MH08 sample list and eliminating binaries in the same way I did to create stars_FGK_nobinaries, so this cross match catches all of them!

Now check query to get the columns that I actually want to put in the ages table.

NOTES
- Sec 4.4 states that tau2 are the preferred ages and that the conservative error is 50%, so I will directly put the error in now.
- Found bibcode using ADS: 2008ApJ...687.1264M

In [35]:
cmd = '''
SELECT mh08_agetable.tau2, 0.5*mh08_agetable.tau2, '2008ApJ...687.1264M', stars_FGK_nobinaries.sid
FROM mh08_agetable
    INNER JOIN stars_FGK_nobinaries 
    ON LTRIM(LTRIM(stars_FGK_nobinaries.hd,'HD')) LIKE mh08_agetable.hd
    OR LTRIM(LTRIM(stars_FGK_nobinaries.GJ,'GJ')) LIKE mh08_agetable.gj
'''
c.execute(cmd)
rows = c.fetchall()
print len(rows)
for row in rows:
    print row

39
(0.8, 0.4, u'2008ApJ...687.1264M', 0)
(5.8, 2.9, u'2008ApJ...687.1264M', 1)
(6.1, 3.05, u'2008ApJ...687.1264M', 2)
(6.9, 3.45, u'2008ApJ...687.1264M', 3)
(6.1, 3.05, u'2008ApJ...687.1264M', 8)
(2.1, 1.05, u'2008ApJ...687.1264M', 9)
(8.9, 4.45, u'2008ApJ...687.1264M', 10)
(5.3, 2.65, u'2008ApJ...687.1264M', 13)
(5.5, 2.75, u'2008ApJ...687.1264M', 18)
(7.1, 3.55, u'2008ApJ...687.1264M', 20)
(7.7, 3.85, u'2008ApJ...687.1264M', 21)
(6.5, 3.25, u'2008ApJ...687.1264M', 27)
(6.1, 3.05, u'2008ApJ...687.1264M', 30)
(4.3, 2.15, u'2008ApJ...687.1264M', 36)
(5.9, 2.95, u'2008ApJ...687.1264M', 35)
(5.9, 2.95, u'2008ApJ...687.1264M', 31)
(2.6, 1.3, u'2008ApJ...687.1264M', 40)
(0.6, 0.3, u'2008ApJ...687.1264M', 42)
(4.8, 2.4, u'2008ApJ...687.1264M', 49)
(4.2, 2.1, u'2008ApJ...687.1264M', 53)
(8.8, 4.4, u'2008ApJ...687.1264M', 48)
(4.4, 2.2, u'2008ApJ...687.1264M', 50)
(0.2, 0.1, u'2008ApJ...687.1264M', 60)
(6.8, 3.4, u'2008ApJ...687.1264M', 72)
(3.6, 1.8, u'2008ApJ...687.1264M', 66)
(2.5, 1.25, u'

## Add rows to the ages table

In [10]:
cmd = '''
INSERT INTO ages (age, age_lo, age_hi, bibcode, sid)
SELECT mh08_agetable.tau2, mh08_agetable.tau2-0.5*mh08_agetable.tau2, mh08_agetable.tau2+0.5*mh08_agetable.tau2,
'2008ApJ...687.1264M', stars_FGK_nobinaries.sid
FROM mh08_agetable
    INNER JOIN stars_FGK_nobinaries 
    ON LTRIM(LTRIM(stars_FGK_nobinaries.hd,'HD')) LIKE mh08_agetable.hd
    OR LTRIM(LTRIM(stars_FGK_nobinaries.GJ,'GJ')) LIKE mh08_agetable.gj
'''
c.execute(cmd)

Check result!

In [11]:
cmd = '''
SELECT * FROM ages
'''
c.execute(cmd)
c.fetchall()

[(1, 0.8, 0.4, 1.2000000000000002, u'2008ApJ...687.1264M', 0),
 (2, 5.8, 2.9, 8.7, u'2008ApJ...687.1264M', 1),
 (3, 6.1, 3.05, 9.149999999999999, u'2008ApJ...687.1264M', 2),
 (4, 6.9, 3.45, 10.350000000000001, u'2008ApJ...687.1264M', 3),
 (5, 6.1, 3.05, 9.149999999999999, u'2008ApJ...687.1264M', 8),
 (6, 2.1, 1.05, 3.1500000000000004, u'2008ApJ...687.1264M', 9),
 (7, 8.9, 4.45, 13.350000000000001, u'2008ApJ...687.1264M', 10),
 (8, 5.3, 2.65, 7.949999999999999, u'2008ApJ...687.1264M', 13),
 (9, 5.5, 2.75, 8.25, u'2008ApJ...687.1264M', 18),
 (10, 7.1, 3.55, 10.649999999999999, u'2008ApJ...687.1264M', 20),
 (11, 7.7, 3.85, 11.55, u'2008ApJ...687.1264M', 21),
 (12, 6.5, 3.25, 9.75, u'2008ApJ...687.1264M', 27),
 (13, 6.1, 3.05, 9.149999999999999, u'2008ApJ...687.1264M', 30),
 (14, 4.3, 2.15, 6.449999999999999, u'2008ApJ...687.1264M', 36),
 (15, 5.9, 2.95, 8.850000000000001, u'2008ApJ...687.1264M', 35),
 (16, 5.9, 2.95, 8.850000000000001, u'2008ApJ...687.1264M', 31),
 (17, 2.6, 1.3, 3.900000

In [13]:
cmd = '''
SELECT s.main_id, a.age, a.age_lo, a.age_hi
FROM stars_FGK_nobinaries AS s
INNER JOIN ages AS a ON a.sid = s.sid
'''
c.execute(cmd)
rows = c.fetchall()
print len(rows)
for row in rows:
    print row

39
(u'* eps Eri', 0.8, 0.4, 1.2000000000000002)
(u'* tau Cet', 5.8, 2.9, 8.7)
(u'* e Eri', 6.1, 3.05, 9.149999999999999)
(u'* del Pav', 6.9, 3.45, 10.350000000000001)
(u'*  61 Vir', 6.1, 3.05, 9.149999999999999)
(u'* zet Tuc', 2.1, 1.05, 3.1500000000000004)
(u'HD 192310', 8.9, 4.45, 13.350000000000001)
(u'HD 103095', 5.3, 2.65, 7.949999999999999)
(u'* alf Men', 5.5, 2.75, 8.25)
(u'HD 166620', 7.1, 3.55, 10.649999999999999)
(u'*  54 Psc', 7.7, 3.85, 11.55)
(u'HD  72673', 6.5, 3.25, 9.75)
(u'HD  69830', 6.1, 3.05, 9.149999999999999)
(u'HD  14412', 4.3, 2.15, 6.449999999999999)
(u'HD 158633', 5.9, 2.95, 8.850000000000001)
(u'HD 104304', 5.9, 2.95, 8.850000000000001)
(u'HD 170657', 2.6, 1.3, 3.9000000000000004)
(u'*  58 Eri', 0.6, 0.3, 0.8999999999999999)
(u'HD 154577', 4.8, 2.4, 7.199999999999999)
(u'*  10 Tau', 4.2, 2.1, 6.300000000000001)
(u'HD 145417', 8.8, 4.4, 13.200000000000001)
(u'*  47 UMa', 4.4, 2.2, 6.6000000000000005)
(u'V* pi.01 UMa', 0.2, 0.1, 0.30000000000000004)
(u'HD 14462

## Check with stars are missing from the MH08 sample

Sample should (!) be complete within 16 pc for stars with spectral types F7-K2 (0.5 < B-V < 0.9). Is this the case or are there many stars in the <tt>stars_FGK_nobinaries</tt> table that fit these criteria but do not have ages reported in MH08?

In [14]:
cmd = '''
SELECT s.main_id, 1/(s.plx_value*1e-3) as d, s.sp_type, s.wds
FROM stars_FGK_nobinaries AS s
LEFT OUTER JOIN ages AS a ON a.sid = s.sid
WHERE d < 16
AND a.ageid IS NULL
AND s.sp_type LIKE '%K%'
'''
c.execute(cmd)
rows = c.fetchall()
print len(rows)
for row in rows:
    print row

52
(u'HD 219134', 6.5324978704056935, u'K3V', u'WDS J23133+5710A')
(u'HD 157881', 7.714674313451846, u'K7V', None)
(u'HD 217357', 8.231008183268337, u'K7+Vk', None)
(u'HD  32147', 8.84770968187175, u'K3+V', None)
(u'* del Eri', 9.040773890245005, u'K0+IV', None)
(u'HD 151288', 9.85109090980735, u'K7.5Ve', None)
(u'HD 122064', 10.077749840015722, u'K3V', None)
(u'HD 103932', 10.184253514585889, u'K4+V', None)
(u'BD+36  2393', 10.958675928939561, u'K7.5Ve', None)
(u'HD  74576', 11.18603267216423, u'K3V', None)
(u'HD  85512', 11.284669325334761, u'K6Vk:', None)
(u'V* V2689 Ori', 11.436845169133784, u'K6V', None)
(u'HD 222237', 11.450740061330162, u'K3+V', None)
(u'HD 125072', 11.840835489352129, u'K3IV', None)
(u'HD 196877', 12.327416173570018, u'K7V', None)
(u'HD  21531', 12.493753123438278, u'K5V', None)
(u'HD 190007', 12.718795072230035, u'K5V', None)
(u'HD 101581', 12.782819890067747, u'K4.5Vk:', None)
(u'HD  82106', 12.782983292640836, u'K3V', None)
(u'HD  40307', 12.941181038063895,

Stars in simbad table and not in MH08, but still in the F7 - K2 spectral type range:
- (u'HD  84117', 14.821555878006736, u'F9V', None): B - V = 5.47 - 4.94 = 0.53 (edge of range?)
- (u'\* gam Pav', 9.261831990367694, u'F9VFe-1.4CH-0.7', None): B-V just outside range
- (u'\* bet Hyi', 7.458790184232118, u'G0V', None): B-V = 0.62? so not sure why this is excluded.
- (u'\* del Eri', 9.040773890245005, u'K0+IV', None): B-V just outside range
- (u'HD  23356', 13.95502017895918, u'K2V', None): B-V just outside range

For stars that I'm not sure about (HD 84117 and \* bet Hyi), I checked B-V using the references listed in Table 13 (#1/2) of MH08 and found consist results with what Simbad reports for B/V magnitudes.

Also checked spectral types with some of the references in Table 13 (#8/11) of MH08 and also found consistent results. So I'm not sure off-hand why these were excluded from the MH08 sample.

In [17]:
connection.commit()

ProgrammingError: Cannot operate on a closed database.

In [18]:
connection.close()